# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    16-Jan-2026**

**News Source / Catalyst:** *Scott Bessent warns South Korean won’s weakness ‘not in line’ with economy* AND *German economy grows for first time since 2022*

**Markets (s) Impacted:** FX

**Trade Taxonomy:** RV Macro

**Trade Idea (1 sentence):**
> Short the structurally impaired EUR against the cyclical KRW to capitalise on Germany's transition to high-deficit stagflation while capturing a US-backed valuation floor ad the AI driven semiconductor export boom in South Korea.

**Client/Desk Pitch (30 seconds):**
> We are recommending a relative value play: Short EUR / Lond KRW. Germany has pivoted to a debt-funded 'regime shift' that will spike deficits to 3.9% of GDP amidst a 3-year manufacturing decline. Conversely, the KRW is at 17-year lows despite a massive AI-chip export surge. With US Treasury Secretary Bassent providing a 'verbal floor' by calling the Won significantly undervalued, we have a unique window to short a decaying fiscal story in Europe and buy a politically-supported, tech-heavy recovery in Asia.

**Why Now:**
- Bessent: Rare US Treasury intervention creats an immediate psychological floor for the Won at the KRW 1,470-1,480 level
- German Fiscal Pivot: Chancellor Merz's January budget confirmation of a fiscal deficit marks the end of German fiscal discipline
- AI Export Peak: South Korean semiconductor exports just hit record highs, creating a fundamental decoupling from the currently depressed currency price

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- Structural Divergence: Germany is entering a 'debt-funde stagnation' phase, while South Korea is entering a 'high-growth tech recovery'
- Political Re-alignment: US support for the Won reduces the risk of further depreciation, effectively lowering the cost of the long leg
- Capital Flow Reversal: Seoul's new tax breaks are designed to force domestic retail investors to repatriate capital from US tech back to the Won

**Long Leg (Tickers + Rationale):**
- **EWY** (Proxy for the Won + AI-driven semi-conductor export boom)

**Short Leg (Tickers + Rationale):**
- **FXE** (Proxy for the Euro; captures the impact of German industrial decay)

**Weights:** Beta-weighted (to account for EWY's higher equity volatility)

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |        S          |       |
| Oil Beta             |        W          |       |
| Rates Duration       |        M          |       |
| Growth vs Value      |        S          |       |
| Risk-On/Off          |        M          |       |
| China/EM Beta        |        M          |       |
| Volatility Regime    |        M          |       |

**Catalyst Type:** Regime Shift 

**Realisation Window:** Weeks

**Holding Period Selected:**  60 Days.

**Alignment Score:** Good


---


## 4. Risk Controls & Stop Logic

**Position Sizing:** For a USD 100,000 Portfolio:
- Long EWY: 225 Units (~USD 25k notional)
- Short FXE: 700 Units (~USD 75k notional)

**Stop-Loss:** -3%

**Take-Profit (optional):**  8%

**Re-entry:** No

**FX-Hedging Required:** Yes (EWY is unhedged equity; EXE is the direct currency short)

**Liquidity Notes:** High (Trade during US/London overlap)  

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information

---


# **PYTHON IMPLEMENTATION**

In [1]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [2]:
# --- Parameters ---

long_tickers = ["EWY"]
short_tickers = ["FXE"]                 

start_date = "2020-01-01"                         # Backtest window
holding_period = 60                               # days
stop_loss = -0.03                                 # -2% stop
take_profit = 0.08                                # optional e.g. +0.03 for +3%
long_positioning = 225
short_positioning = 700

# --- Weightings ---

weights_long = long_positioning / (long_positioning + short_positioning)
weights_short = short_positioning / (long_positioning + short_positioning)

print(weights_long, weights_short)


0.24324324324324326 0.7567567567567568


In [3]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_30114/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  2 of 2 completed


In [4]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [5]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
20,2024-10-10,-0.0300
21,2025-01-07,-0.0487
22,2025-04-04,-0.0304
23,2025-07-02,0.0900
24,2025-09-26,0.0963


In [6]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         25.0000
Win Rate              0.3200
Average PnL           0.0014
Best Trade            0.1063
Worst Trade          -0.0624
Stop-Out Frequency    0.6800
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 
- Win Rate:
- Avg PnL:
- Best:
- Worst: 
- Stop-Outs: 

**What went right:**
- 

**What went wrong:**
- 

**Factor that actually drove results:**
- 

**Would I trade this live?** (Yes / No and why)
>

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [7]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [8]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [9]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
20,RV-211361F1,2026-01-16 10:08:06,Macro RV Spread,RV Macro Spread,EWY,FXE,2024-10-10,2024-10-22,-60.6649,-60.3107,-3.0047,-3.0000,8.0000,STOPPED,12,
21,RV-8C268D1B,2026-01-16 10:08:06,Macro RV Spread,RV Macro Spread,EWY,FXE,2025-01-07,2025-03-10,-58.8388,-62.3393,-4.8696,-3.0000,8.0000,STOPPED,62,
22,RV-98403000,2026-01-16 10:08:06,Macro RV Spread,RV Macro Spread,EWY,FXE,2025-04-04,2025-04-04,-63.8137,-63.8137,-3.0443,-3.0000,8.0000,STOPPED,0,
23,RV-9B7BFD92,2026-01-16 10:08:06,Macro RV Spread,RV Macro Spread,EWY,FXE,2025-07-02,2025-09-25,-65.0553,-62.2688,12.5854,-3.0000,8.0000,TAKE_PROFIT,85,
24,RV-BBAD0734,2026-01-16 10:08:06,Macro RV Spread,RV Macro Spread,EWY,FXE,2025-09-26,2025-12-19,-62.8362,-59.5497,16.1356,-3.0000,8.0000,TAKE_PROFIT,84,


In [10]:
blotter.to_csv("trade_blotter.csv", index=False)